<a href="https://colab.research.google.com/github/yhp2205/SQL/blob/main/ch_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 8강 여러 개의 테이블 조작하기

In [2]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### 여러 개의 테이블을 세로로 결합

세로로 결합하기 위한 데이터 테이블을 app1_mst_users와 app2_mst_users로 나누어 작성하겠습니다. 

In [3]:
%%bigquery --project mygcpproject-340112
DROP TABLE IF EXISTS sqldata.app1_mst_users;
CREATE TABLE sqldata.app1_mst_users (
    user_id string
  , name    string
  , email   string
);

INSERT INTO sqldata.app1_mst_users
VALUES
    ('U001', 'Sato'  , 'sato@example.com'  )
  , ('U002', 'Suzuki', 'suzuki@example.com')
;

DROP TABLE IF EXISTS sqldata.app2_mst_users;
CREATE TABLE sqldata.app2_mst_users (
    user_id string
  , name    string
  , phone   string
);

INSERT INTO sqldata.app2_mst_users
VALUES
    ('U001', 'Ito'   , '080-xxxx-xxxx')
  , ('U002', 'Tanaka', '070-xxxx-xxxx')
;


""


두 데이터의 구조가 user_id, name은 동일하고 phone과 email 등의 컬럼이 다른 비슷한 구조를 지니고 있기 때문에 UNION ALL 구문을 사용하여 여러 개의 테이블을 세로로 결합할 수 있습니다. 한쪽 테이블에만 존재하는 컬럼은 SELECT 구문에서 제외하거나, 디폴트 값을 줌으로써 처리하면 됩니다.

In [4]:
%%bigquery --project mygcpproject-340112
  -- 8.1 UNION ALL 구문을 사용해 테이블을 세로로 결합하는 쿼리
  SELECT 'app1' AS app_name, user_id, name, email FROM sqldata.app1_mst_users
UNION ALL
  SELECT 'app2' AS app_name, user_id, name, NULL AS email FROM sqldata.app2_mst_users;

,app_name,user_id,name,email
0,app2,U001,Ito,None
1,app2,U002,Tanaka,None
2,app1,U001,Sato,sato@example.com
3,app1,U002,Suzuki,suzuki@example.com


다음과 같이 phone 컬럼을 제외하고, email 컬럼에 디폴트 값으로 NULL 값을 지정함으로써 두 테이블을 세로로 결합해보았습니다. 

### 여러 개의 테이블을 가로로 정렬

In [5]:
%%bigquery --project mygcpproject-340112
DROP TABLE IF EXISTS sqldata.mst_categories;
CREATE TABLE sqldata.mst_categories (
    category_id integer
  , name        string
);

INSERT INTO sqldata.mst_categories
VALUES
    (1, 'dvd' )
  , (2, 'cd'  )
  , (3, 'book')
;

DROP TABLE IF EXISTS sqldata.category_sales;
CREATE TABLE sqldata.category_sales (
    category_id integer
  , sales       integer
);

INSERT INTO sqldata.category_sales
VALUES
    (1, 850000)
  , (2, 500000)
;

DROP TABLE IF EXISTS sqldata.product_sale_ranking;
CREATE TABLE sqldata.product_sale_ranking (
    category_id integer
  , rank        integer
  , product_id  string
  , sales       integer
);

INSERT INTO sqldata.product_sale_ranking
VALUES
    (1, 1, 'D001', 50000)
  , (1, 2, 'D002', 20000)
  , (1, 3, 'D003', 10000)
  , (2, 1, 'C001', 30000)
  , (2, 2, 'C002', 20000)
  , (2, 3, 'C003', 10000)
;


""


여러 개의 테이블을 가로로 정렬할 때에는 JOIN을 사용하면 편리합니다.

In [6]:
%%bigquery --project mygcpproject-340112
  -- 8.2 여러 개의 테이블을 결합해서 가로로 정렬하는 쿼리
SELECT
    m.category_id
    , m.name
    , s.sales
    , r.product_id AS sale_product
FROM
    sqldata.mst_categories AS m
  JOIN
    -- 카테고리별로 매출액 결합하기
    sqldata.category_sales AS s
    ON m.category_id = s.category_id
  JOIN
    -- 카테고리별로 상품 결합하기
    sqldata.product_sale_ranking AS r
    ON m.category_id = r.category_id
;

,category_id,name,sales,sale_product
0,2,cd,500000,C001
1,2,cd,500000,C002
2,2,cd,500000,C003
3,1,dvd,850000,D001
4,1,dvd,850000,D002
5,1,dvd,850000,D003


단순히 category_id로 테이블을 결합한 결과, book 카테고리가 결합되지 못하고 여러 상품 데이터가 소실된 것을 볼 수 있습니다.  
마스터 테이블의 행 수를 변경하지 않고 데이터를 가로 정렬하기 위해서는 LEFT JOIN을 사용하여 결합하지 못한 레코드를 유지하면서, 결합할 레코드가 1개 이하가 되도록 조건을 지정해야 합니다. 

In [7]:
%%bigquery --project mygcpproject-340112
  -- 8.3 마스터 테이블의 행 수를 변경하지 않고 여러 개의 테이블을 가로로 정렬하는 쿼리
SELECT
  m.category_id
  , m.name
  , s.sales
  , r.product_id AS top_sale_product
FROM
    sqldata.mst_categories AS m
  -- LEFT JOIN 을 사용해서 결합한 레코드를 남김
  LEFT JOIN
    -- 카테고리별 매출액 결합하기
    sqldata.category_sales AS s
    ON m.category_id = s.category_id
  -- LEFT JOIN을 사용해서 결합하지 못한 레코드를 남김
  LEFT JOIN
    -- 카테고리별 최고 매출 상품 하나만 추출해서 결합하기
    sqldata.product_sale_ranking AS r
    ON m.category_id = r.category_id
    AND r.rank = 1
;

,category_id,name,sales,top_sale_product
0,2,cd,500000.0,C001
1,1,dvd,850000.0,D001
2,3,book,NaN,None


앞서 코드에서 rank = 1 을 지정하여 매출 순위의 1위 상품이라는 조건을 걸어 카테고리가 여러 행이 되지 않도록 하였습니다.  
  
다음 코드는 JOIN을 사용하지 않고 앞과 같은 결과를 출력하도록 하는 쿼리입니다.

In [8]:
%%bigquery --project mygcpproject-340112
  --  8.4 상관 서브쿼리로 여러 개의 테이블을 가로로 정렬하는 쿼리
SELECT
    m.category_id
  , m.name
    -- 상관 서브 쿼리를 사용해 카테고리별로 매출액 추출하기
  , (SELECT s.sales
     FROM sqldata.category_sales AS s
     WHERE m.category_id = s.category_id
     ) AS sales
     -- 상관 서브쿼리를 사용해 카테고리별로 최고 매출 상품을
     -- 하나 추출하기(순위로 따로 압축하지 않아도 됨)
  , (SELECT r.product_id
     FROM sqldata.product_sale_ranking AS r
     WHERE m.category_id = r.category_id
     ORDER BY sales DESC
     LIMIT 1
     ) AS top_sale_product
FROM
    sqldata.mst_categories AS m
;

Executing query with job ID: 773089f8-43d4-40f5-9297-1b23b93d5dcb
Query executing: 0.25s


ERROR:
 400 Correlated subqueries that reference other tables are not supported unless they can be de-correlated, such as by transforming them into an efficient JOIN.

(job ID: 773089f8-43d4-40f5-9297-1b23b93d5dcb)

        -----Query Job SQL Follows-----         

    |    .    |    .    |    .    |    .    |
   1:--  8.4 상관 서브쿼리로 여러 개의 테이블을 가로로 정렬하는 쿼리
   2:SELECT
   3:    m.category_id
   4:  , m.name
   5:    -- 상관 서브 쿼리를 사용해 카테고리별로 매출액 추출하기
   6:  , (SELECT s.sales
   7:     FROM sqldata.category_sales AS s
   8:     WHERE m.category_id = s.category_id
   9:     ) AS sales
  10:     -- 상관 서브쿼리를 사용해 카테고리별로 최고 매출 상품을
  11:     -- 하나 추출하기(순위로 따로 압축하지 않아도 됨)
  12:  , (SELECT r.product_id
  13:     FROM sqldata.product_sale_ranking AS r
  14:     WHERE m.category_id = r.category_id
  15:     ORDER BY sales DESC
  16:     LIMIT 1
  17:     ) AS top_sale_product
  18:FROM
  19:    sqldata.mst_categories AS m
  20:;
    |    .    |    .    |    .    |    .    |


In [9]:
%%bigquery --project mygcpproject-340112
DROP TABLE IF EXISTS sqldata.mst_users_with_card_number;
CREATE TABLE sqldata.mst_users_with_card_number (
    user_id     string
  , card_number string
);

INSERT INTO sqldata.mst_users_with_card_number
VALUES
    ('U001', '1234-xxxx-xxxx-xxxx')
  , ('U002', NULL                 )
  , ('U003', '5678-xxxx-xxxx-xxxx')
;

DROP TABLE IF EXISTS sqldata.purchase_log;
CREATE TABLE sqldata.purchase_log (
    purchase_id integer
  , user_id     string
  , amount      integer
  , stamp       string
);

INSERT INTO sqldata.purchase_log
VALUES
    (10001, 'U001', 200, '2017-01-30 10:00:00')
  , (10002, 'U001', 500, '2017-02-10 10:00:00')
  , (10003, 'U001', 200, '2017-02-12 10:00:00')
  , (10004, 'U002', 800, '2017-03-01 10:00:00')
  , (10005, 'U002', 400, '2017-03-02 10:00:00')
;


""


In [11]:
%%bigquery --project mygcpproject-340112
  -- 8.5 신용 카드 등록과 구매 이력 유무를 0과 1이라는 플래그로 나타내는 쿼리
SELECT
  m.user_id
  , m.card_number
  , COUNT(p.user_id) AS purchase_count
    -- 신용 카드 번호를 등록한 경우 1, 등록하지 않은 경우 0으로 표현하기
  , CASE WHEN m.card_number IS NOT NULL THEN 1 ELSE 0 END AS has_card
    -- 구매 이력이 있는 경우 1, 없는 경우 0으로 표현하기
  , SIGN(COUNT(p.user_id)) AS has_purchased
FROM
    sqldata.mst_users_with_card_number AS m
  LEFT JOIN
    sqldata.purchase_log AS p
    ON m.user_id = p.user_id
GROUP BY m.user_id, m.card_number
;

,user_id,card_number,purchase_count,has_card,has_purchased
0,U003,5678-xxxx-xxxx-xxxx,0,1,0
1,U002,None,2,0,1
2,U001,1234-xxxx-xxxx-xxxx,3,1,1
